# search engine with tools and agents

In [2]:
# Arxiv--Research 
# Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [11]:
#used the inbuild tools of wikipedia
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [12]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_result=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper_arxiv=api_wrapper_arxiv)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))

In [15]:
tools=[wiki,arxiv]

In [17]:
import os
from dotenv import load_dotenv
load_dotenv() # load all environment variables
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [14]:
## custom tools RAG Tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [19]:
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
loader=WebBaseLoader("https://python.langchain.com/docs/introduction/?_gl=1*16mun4n*_gcl_au*NTM5MDQ1OTA1LjE3NTkwMzY1Mzg.*_ga*MzcyMDU5MzM5LjE3NTkwMzY1Mzg.*_ga_47WX3HKKY2*czE3NjAyNzE2MjMkbzkkZzAkdDE3NjAyNzE2MjMkajYwJGwwJGgw")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embedding)
retriever=vectordb.as_retriever()

In [20]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langchain-search","search any informaton about langsmith")

retriever_tool.name

'langchain-search'

In [21]:
tools=[wiki,arxiv,retriever_tool]

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\AI\\langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langchain-search', description='search any informaton about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000180AA7462A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000180CFF7B5F0>, search_kw

In [24]:
## Run all this tools with agents and llm models 

## tools,LLM-->agengtsExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

import os

groq_api_key=os.getenv('GROQ_API_KEY')
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

In [ ]:
## prompt Template
from smolagents import CodeAgent

system_prompt = """
You are a helpful AI assistant that can use external tools.
You can call tools to perform tasks such as math or data operations.
When you use a tool, explain your reasoning briefly first.
Always return a clear final answer.
"""
agent = CodeAgent(
    model=llm,                # equivalent to 'llm' in LangChain
    tools=tools,              # list of Tool objects
    system_prompt=system_prompt,  # equivalent to the pulled prompt
    max_steps=4, 
    verbose=True   # will print reasoning steps: which tool was called, the input/output, etc.
)

question="what is machine learing "
result=agent.run(question)